In [1]:
#IMPORT LIBRARIES
#Try to model blackjack outcomes long term
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#RANDOMIZE DRAWING CARDS
test =[]
card_options = [2,3,4,5,6,7,8,9,10,10,10,10,11]
for deals in range(1000):
    deals = random.choice(card_options)
    test.append(deals)


In [3]:
#VERIFY CARD PROBABILITIES ARE CORRECT

results = pd.DataFrame(test, columns = ["Card Values"])
#results["Card Values"].value_counts().plot(kind="bar")

In [65]:
#DEFINE ALL FUNCTIONS
def hit_me(current_total, player = "True"): #I use this function to model dealer hand too and I don't want hit_count to go up
    global thrid_card_list
    global forth_card_list
    global fifth_card_list
    global sixth_card_list
    
    another_card = random.choice(card_options)
    if another_card == 11 and current_total >=11: #ace only worth 1 if total goes over 21 (current total = 11)
        another_card = 1
    else:
        another_card = another_card
    global hit_count #have to define this as global or else these variables or only defined within the function. I want the rest of my code to use it also
    if player == "True":
        hit_count = hit_count + 1

    if hit_count == 1: #for analysis purposes to record additional cards drew
        thrid_card_list.append(another_card)
    elif hit_count == 2:
        forth_card_list.append(another_card)
    elif hit_count == 3:
        fifth_card_list.append(another_card)
    elif hit_count == 4:
        sixth_card_list.append(another_card)

    current_total = current_total + another_card
    return current_total

#define stay function
def stay(current_total):
    global hit
    global soft
    hit = False
    soft = False
    return current_total
    
#can't double down if already taken a card. Can have either (double or hit) or (double or stay). Default is to hit if cannot double
def double_down(current_total, Ds = "Hit"):
    global hit_count
    global bet
    global hit
    global soft
    global double
    if hit_count >= 1 and Ds == "Hit": #hit
        current_total = hit_me(current_total)
    elif hit_count >= 1 and Ds == "Stay": #stay
        current_total = stay(current_total)
    else: #double
        bet = 2*bet
        current_total = hit_me(current_total)
        hit = False
        soft = False
        double = True
    return current_total

#compare player vs dealer's hand
def compare_hands(player_total, dealer_total, first_card, second_card, dealer_first_card, dealer_second_card):
    global current_money
    global bet
    global blackjack

    #player bust
    if player_total > 21:
        current_money = current_money - bet
        outcome = "Bust"
    #blackjack
    elif (first_card + second_card) == 21:
        if (dealer_first_card + dealer_second_card) == 21:
            current_money = current_money
            outcome = "Push"
        else:
            current_money = current_money + blackjack*bet
            outcome = "Blackjack"
    #dealer bust
    elif dealer_total > 21 and player_total <= 21:
        current_money = current_money + bet
        outcome = "Dealer Bust"
    #win
    elif player_total > dealer_total:
        current_money = current_money + bet
        outcome = "Win"
    #loss
    elif player_total < dealer_total:
        current_money = current_money - bet
        outcome = "Loss"
    #push
    elif player_total == dealer_total:
        current_money = current_money
        outcome = "Push"
    else:
        outcome = "Undefined"

    return [current_money, outcome]
 
def soft_totals(current_total, dealer_first_card):
    global hit
    global soft
    global hit_count
    while soft == True:                
        #if soft total >= 19, stay
        if (current_total >= 19):
            current_total = stay(current_total)
        #if soft total = 18 and dealer has <=6, double
        elif (current_total == 18) and dealer_first_card <= 6:
            current_total = double_down(current_total, Ds="Stay")
            #print(current_total, first_card, second_card, dealer_first_card)
        #if soft total = 18 and dealer has 7 or 8, stay
        elif (current_total == 18) and (dealer_first_card == 7 or dealer_first_card == 8):
            current_total = stay(current_total)
        #if soft total = 18 and dealer has >= 9, hit
        elif (current_total == 18) and (dealer_first_card >= 9):
            current_total = hit_me(current_total)
        #if soft total = 17 and dealer has 3 - 6, double
        elif (current_total == 17) and (dealer_first_card >= 3 and dealer_first_card <= 6):
            current_total = double_down(current_total)
        #if soft total = 17 and dealer has 2 or >=7, hit
        elif (current_total == 17) and (dealer_first_card == 2 or dealer_first_card >= 7):
            current_total = hit_me(current_total)
        #if soft total = 15-16 and dealer has 4 - 6, double
        elif (current_total == 15 or current_total == 16) and (dealer_first_card >= 4 and dealer_first_card <= 6):
            current_total = double_down(current_total)
        #if soft total = 15-16 and dealer has 2 or >= 7, hit
        elif (current_total == 15 or current_total == 16) and (dealer_first_card <= 3 or dealer_first_card >= 7):
            current_total = hit_me(current_total)
        #if soft total = 13-15 and dealer has 5-6, double
        elif (current_total == 13 or current_total == 14) and (dealer_first_card == 5 or dealer_first_card == 6):
            current_total = double_down(current_total)
        #if soft total = 13-15 and dealer has <=4 or >= 7, hit
        elif (current_total == 13 or current_total == 14) and (dealer_first_card <= 4 or dealer_first_card >= 7):
            current_total = hit_me(current_total)
        else:
            player_total = 70
            hit = False
            soft = False
    
        #######################
        #exit soft totals loop if gone over 21. Re-enter hardtotals loop
        if current_total > 21:
            current_total = current_total - 10
            soft = False
            #break
            #hit = False

    else: #need the else statement (in line with while loop because player_total is a function output and may not be defined in the if/elif statments, meaning the function can't return (for example when hitting) anything for this variable
        player_total = current_total #player total can equal current total with no harm done up until comparing to dealer's hand. By then all possible hands would have defined player_total as final value (written explicitly after stay and double function)
    return [player_total, current_total] #need to return player_total & current total because some hands continue after this loop and some stop, so both are important

def hard_totals(current_total, dealer_first_card):
    global soft #don't need soft here
    global hit
    global hit_count
    #global player_total
    while hit == True:
        #if hard total >=17, stay (also bust)
        if current_total >= 17:
            current_total = stay(current_total)
        #if hard total is 13-16 dealer <= 6, stay
        elif current_total <= 16 and current_total >= 13 and dealer_first_card <= 6:
            current_total = stay(current_total)
        #if hard total is 13-16 dealer > 6, hit
        elif current_total <= 16 and current_total >= 13 and dealer_first_card > 6:
            current_total = hit_me(current_total)
        #if hard total is 12 dealer = 2,3, >= 7, hit
        elif current_total == 12 and (dealer_first_card <= 3 or dealer_first_card >= 7):
            current_total = hit_me(current_total)
        #if hard total is 12 dealer = 4,5,6, stay
        elif current_total == 12 and (dealer_first_card >= 4 and dealer_first_card <= 6):
            current_total = stay(current_total)
        #if hard total = 11, double
        elif current_total == 11:
            current_total = double_down(current_total)
        #if hard total = 10 & dealer <=9, double
        elif current_total == 10 and dealer_first_card <= 9:
            current_total = double_down(current_total)
        #if hard total = 10 & dealer >9, hit
        elif current_total == 10 and dealer_first_card > 9:
            current_total = hit_me(current_total)
        #if hard total = 9 & dealer 3-6, double
        elif current_total == 9 and (dealer_first_card >= 3 and dealer_first_card <= 6):
            current_total= double_down(current_total)
        #if hard total = 9 & dealer =2 or >=7, hit
        elif current_total == 9 and (dealer_first_card == 2 or dealer_first_card >=7):
            current_total = hit_me(current_total)
        #if hard total <=8, hit
        elif current_total <= 8: 
            current_total = hit_me(current_total)
        else:
            player_total = 60
            hit = False
    else: #need the else statement (in line with while loop because player_total is a function output and may not be defined in the if/elif statments, meaning the function can't return (for example when hitting) anything for this variable
          player_total = current_total #player total can equal current total with no harm done up until comparing to dealer's hand. By then all possible hands would have defined player_total as final value (written explicitly after stay and double function)
    return player_total

def splits(current_total, first_card, dealer_first_card):
    global split
    global hit
    global soft
    global hit_count
    global bet
    global blackjack
    global double
    #define when to split and not
    if (first_card == 2 or first_card == 3) and dealer_first_card <=7:
        split = True
    elif (first_card == 2 or first_card == 3) and dealer_first_card > 7:
        split = False
    elif (first_card == 4) and (dealer_first_card == 5 or dealer_first_card == 6):
        split = True
    elif (first_card == 4) and (dealer_first_card <= 4 or dealer_first_card >= 7):
        split = False
    elif first_card == 5 or first_card == 10:
        split = False
    elif first_card == 6 and dealer_first_card <=6:
        split = True
    elif first_card == 6 and dealer_first_card >=7:
        split = False
    elif first_card == 7 and dealer_first_card <=7:
        split = True
    elif first_card == 7 and dealer_first_card >=8:
        split = False
    elif first_card == 8 or first_card == 11:
        split = True
    elif first_card == 9 and (dealer_first_card == 7 or dealer_first_card >=10):
        split = False
    elif first_card == 9 and (dealer_first_card <=6 or dealer_first_card == 8 or dealer_first_card == 9):
        split = True
    else:
        split = False
        current_total = 40
        player_total = 40

    #if not splitting, return to hard totals loop
    if split == False:
        player_total = hard_totals(current_total, dealer_first_card)
        double_list_local = double
        bet_list_local = bet
        list_of_dictionaries = {"player_total": player_total,"bet": bet, "double": double, "hit count": hit_count}
        
        
    #If splitting, model the 2 hands seperatly
    elif split == True:
        list_of_dictionaries = []
        n_hands = ["hand 1", "hand 2"]

        if random.choice(card_options) == first_card and first_card != 11:  #model 3rd split by adding item to n_hands list. Has to be before for loop, so this just models whether a 3rd card will be the same as current. Can't split aces twice
                n_hands.append("hand 3")
                if random.choice(card_options) == first_card:  #model 3rd split
                    n_hands.append("hand 4")

        for hand in range(len(n_hands)):  #loop though and model each split hand a # of times equal to the length of n_hands list

            hand_total = hit_me(first_card) #first card is current total. This gives the hand 2 cards 
            hit_count = 0 #reset hit_count for each hand
            hit = True #reset initial variables or else hit=False for next hand
            soft=True
            bet = 5
            double = False 


            #can only recieve 1 more card if splitting aces
            if first_card == 11:
                hand_total = stay(hand_total)

            #IF 2ND CARD IS AN ACE, RUN SOFTOTALS->HARDTOTALS, IF NOT AN ACE RUN HARDTOTALS
            elif (hand_total - first_card) == 11: 
                soft_function = soft_totals(hand_total, dealer_first_card)
                hand_total = soft_function[0] #redefine function outputs because they're a list and global variables doens't work here
                hand_total = soft_function[1] #same as above

                if hit != False:  #only run hardtotals function if you didn't stay or double in soft total loop (you hit and went over 21)
                    hand_total = hard_totals(hand_total, dealer_first_card)

            else:
                hand_total = hard_totals(hand_total, dealer_first_card)

            #need to have list of dictionaries because more than 1 hands are modeled here and I need to carry bet and double variables with each hand. Before this line of code my bet and hit_count varible wasn't carrying with the hand
            list_of_dictionaries.append({"player_total": hand_total,"bet": bet, "double": double, "hit count": hit_count})

            # #TESTING HOW LIST OF DICTIONARIES WORK
            # totals = [{"player_total": 1,"bet": 5, "Double": False},
            # {"player_total": 2,"bet": 5, "Double": False},
            # {"player_total": 3,"bet": 10, "Double": True}]

            # totals[1]["bet"]

            # for hand in totals:
            # print(hand["bet"])
    else:
        current_total = 87
        player_total = 87

    return list_of_dictionaries

In [186]:
#BLACKJACK MODELING WITH FUNCTIONS

# RULES:
# 1. Dealer stays on soft 17
# 2. No Surrender
# 3. Doubling allowed on all hands before taking a 3rd card
# 4. Doubling after splitting allowed
# 5. No blackjack after splitting. Hand equals 21, player doesn't recieve 1.5*bet
# 6. You cannot re-spit aces (if another ace is dealt after splitting aces you can't split again). Gives house 0.08% more edge
#doesn't model hand as soft if 3rd+ hand is ace (niether for player nor dealer!!) (in hardtotals and split functions), treats it like hard hand, ace could be 11 or 1. Depends if current total >=11


#define input varibles
current_money = 15000
bet = 5
blackjack = 1.5

#define lists to later append data in
money = []
player_total_list = []
dealer_total_list = []
player_1st_list = []
player_2nd_list = []
dealer_1st_list = []
dealer_2nd_list = []
hit_count_list = []
outcome_list = []
double_list = []
split_list = []
bet_list = []
thrid_card_list = []
forth_card_list = []
fifth_card_list = []
sixth_card_list = []

#start for loop to model 100,000 hands of play
for hand in range(1000000):

    #deal cards from infinite deck
    first_card = random.choice(card_options)
    second_card = random.choice(card_options)
    dealer_first_card = random.choice(card_options)
    dealer_second_card = random.choice(card_options)
    current_total = (first_card) + (second_card)
    
    hit = True #go through while loop at least once
    soft = True #go through softtotal while loop at least once if one card is an ace. loop through soft total until staying or going over 21 which exits this loop and goes to 1st loop for hard totals
    double = False #only for analysis to tell weather hand was doubled or not
    split = False #don't compare player's & dealer's hand unless cards were split
    player_total = 0
    dealer_total = 0
    bet = 5
    hit_count = 0


    ###########################################################################################################################################
    #model gampeplay
    ###########################################################################################################################################

    #blackjack
    if (first_card + second_card) == 21:
        player_total = 21
        hit = False
    ###########################################################################################################################################
    #if cards are pairs, SPLIT OR NOT
    elif (first_card) == (second_card):
        split_cards = splits(current_total, first_card, dealer_first_card)
        # player_total = split_cards[0]
        # double = split_cards[1]
        # bet = split_cards[2]
        list_of_dictionaries = split_cards
        #print(list_of_dictionaries)
        if split == False:
            player_total = list_of_dictionaries["player_total"]
            bet = list_of_dictionaries["bet"]
            double = list_of_dictionaries["double"]
            hit_count = list_of_dictionaries["hit count"]

    ###########################################################################################################################################

    ###########################################################################################################################################
    #SOFT TOTALS  -- if 1 card is an ace     
    elif (first_card) == 11 or (second_card) == 11:  #need the soft == True clause to avoid exiting softtotal loop, going through hardtotal loop, and coming back into softotal loop
        
        #run soft_totals function
        soft_function = soft_totals(current_total, dealer_first_card)

        #redefine function outputs because they're a list and global variables doens't work here
        player_total = soft_function[0]
        current_total = soft_function[1]

        #only run hardtotals function if you didn't stay or double in soft total loop (you hit and went over 21)
        if hit != False:  
            player_total = hard_totals(current_total, dealer_first_card)
    ##########################################################################################################################################
    else:
        player_total = hard_totals(current_total, dealer_first_card)

    ##########################################################################################################################################
    #model dealer's hand (dealer stays on soft 17). Need to account for dealer soft hands too
    dealer_total = dealer_first_card + dealer_second_card
    dealer_soft = False #boolean variable telling code whether to model dealer's hand as soft hand or not. Default is hard hand
    if dealer_first_card == 11 or dealer_second_card == 11: #if one of dealers cards is ace model as soft hand
        dealer_soft = True
        if dealer_first_card == 11 and dealer_second_card == 11: #if dealer dealt ace/ace. Other wise if dealt just one ace, model in while loop below
            dealer_total = 12
    while dealer_total < 17:
        dealer_total = hit_me(dealer_total, player = "False")
        if dealer_soft == True and dealer_total > 21:
            dealer_total = dealer_total - 10
            dealer_soft = False
    ##########################################################################################################################################


    #compare hands and append values to lists
    if split == True: #only run this if cards were split, have to be modeled as seperate hands
        #print(list_of_dictionaries)
        for each_hand in list_of_dictionaries:
            #print(each_hand)
            player_total = each_hand["player_total"]
            bet = each_hand["bet"]
            double = each_hand["double"]
            hit_count = each_hand["hit count"]

            hand_results = compare_hands(player_total, dealer_total, first_card, second_card, dealer_first_card, dealer_second_card)
            current_money = hand_results[0]
            outcome = hand_results[1]

            #append variables to list
            money.append(current_money)
            player_total_list.append(player_total)
            dealer_total_list.append(dealer_total)
            player_1st_list.append(first_card)
            player_2nd_list.append(second_card)
            dealer_1st_list.append(dealer_first_card)
            dealer_2nd_list.append(dealer_second_card)
            hit_count_list.append(hit_count)
            outcome_list.append(outcome)
            split_list.append(split)
            bet_list.append(bet)
            double_list.append(double)

    else: #if no split occurred
        hand_results = compare_hands(player_total, dealer_total, first_card, second_card, dealer_first_card, dealer_second_card)
        current_money = hand_results[0]
        outcome = hand_results[1]

        #append variables to list
        money.append(current_money)
        player_total_list.append(player_total)
        dealer_total_list.append(dealer_total)
        player_1st_list.append(first_card)
        player_2nd_list.append(second_card)
        dealer_1st_list.append(dealer_first_card)
        dealer_2nd_list.append(dealer_second_card)
        hit_count_list.append(hit_count)
        outcome_list.append(outcome)
        split_list.append(split)
        bet_list.append(bet)
        double_list.append(double)
        # thrid_card_list = []
        # forth_card_list = []
        # fifth_card_list = []
        # sixth_card_list.append() 

    #reset variables
    player_total = 0
    dealer_total = 0
    bet = 5
    hit_count = 0
  


In [187]:
#CREATE DATAFRAME
df = pd.DataFrame({"Current Money":money,
                   "Player Totals": player_total_list,
                   "Dealer Totals": dealer_total_list,
                   "Player 1st Card": player_1st_list,
                   "Player 2nd Card": player_2nd_list,
                   "Dealer 1st Card": dealer_1st_list,
                   "Dealer 2nd Card": dealer_2nd_list,
                   "Hit Count": hit_count_list,
                   "Bet": bet_list,
                   "Outcome": outcome_list,
                   "Double": double_list,
                   "Split": split_list
                   })
df["Hit Count"].value_counts()

Hit Count
0    486602
1    406418
2    111211
3     21290
4      2635
5       262
6        13
7         2
Name: count, dtype: int64

In [188]:
#VALUE COUNTS OF PLAYER TOTALS
df["Player Totals"].value_counts() # 10 and 11 can still come up if doubling on 9 or 10 and you get a 2

Player Totals
20    166892
19    122578
17    122535
18    116241
21    108955
13     51364
22     48609
15     45641
14     45236
16     42017
23     39160
24     33726
12     32694
25     28956
26     22892
11       937
Name: count, dtype: int64

In [189]:
#OVERALL OUTCOME PERCENTAGES
total_win_percent =(len(df.loc[df["Outcome"] == "Win"])+len(df.loc[df["Outcome"] == "Dealer Bust"])+len(df.loc[df["Outcome"] == "Blackjack"]))/len(df)
total_loss_percent = (len(df.loc[df["Outcome"] == "Loss"])+len(df.loc[df["Outcome"] == "Bust"]))/len(df)
win_only_percent = len(df.loc[df["Outcome"] == "Win"])/len(df)
dealer_bust_percent = len(df.loc[df["Outcome"] == "Dealer Bust"])/len(df)
loss_only_percent = len(df.loc[df["Outcome"] == "Loss"])/len(df)
push_percent = len(df.loc[df["Outcome"] == "Push"])/len(df)
blackjack_percent = len(df.loc[df["Outcome"] == "Blackjack"])/len(df)
bust_percent = len(df.loc[df["Outcome"] == "Bust"])/len(df)

print(f"Overall Win %: {total_win_percent:.2%}")
print(f"Overall Loss %: {total_loss_percent:.2%}")
print(f"Push %: {push_percent:.2%}")
print(f"Win Only %: {win_only_percent:.2%}")
print(f"Dealer Bust %: {dealer_bust_percent:.2%}")
print(f"Loss Only % %: {loss_only_percent:.2%}")
print(f"Blackjack %: {blackjack_percent:.2%}")
print(f"Bust %: {bust_percent:.2%}")

Overall Win %: 43.39%
Overall Loss %: 47.63%
Push %: 8.98%
Win Only %: 15.63%
Dealer Bust %: 23.37%
Loss Only % %: 30.77%
Blackjack %: 4.39%
Bust %: 16.86%


In [190]:
#CREATE HAND ID FOR ALL HAND COMBINATIONS TO GROUPBY
df["Hand1"] = df[["Player 2nd Card", "Player 1st Card"]].min(axis=1) #need "axis=1" for some reason
df["Hand2"] = df[["Player 2nd Card", "Player 1st Card"]].max(axis=1) #need "axis=1" for some reason
df = df.sort_values(by= "Hand1", ascending=True)
df["Hand (1st,2nd / Dealer's)"] = df["Hand1"].astype(str) + "," + df["Hand2"].astype(str) + " / " + df["Dealer 1st Card"].astype(str)
df["Player Hand"] = df["Hand1"].astype(str) + "," + df["Hand2"].astype(str)
df = df.drop(columns=["Hand2", "Hand1"])
df.head(5)



,Current Money,Player Totals,Dealer Totals,Player 1st Card,Player 2nd Card,Dealer 1st Card,Dealer 2nd Card,Hit Count,Bet,Outcome,Double,Split,"Hand (1st,2nd / Dealer's)",Player Hand
1028432,-10425.0,22,21,10,2,11,10,1,5,Bust,False,False,"2,10 / 11","2,10"
543768,2090.0,15,17,10,2,2,10,1,5,Loss,False,False,"2,10 / 2","2,10"
129221,13385.0,20,20,10,2,9,11,1,5,Push,False,False,"2,10 / 9","2,10"
798623,-2222.5,19,19,2,7,3,6,1,10,Push,True,False,"2,7 / 3","2,7"
273139,9802.5,12,20,2,10,5,8,0,5,Loss,False,False,"2,10 / 5","2,10"


In [191]:
#SEPERATE OUTCOMES INTO NEW COLUMNS
win_only = []
overall_win = []
loss_only = []
overall_loss = []
bust = []
dealer_bust = []
blackjack = []
push = []

#run for loop through outcome column to seperate outcomes into new column so I can get a % of each outcome by hand
for outcome in df["Outcome"]:
    if outcome == "Win":
        overall_win.append(outcome)
        overall_loss.append(np.nan)
        win_only.append(outcome)
        loss_only.append(np.nan)
        bust.append(np.nan)
        dealer_bust.append(np.nan)
        blackjack.append(np.nan)
        push.append(np.nan)
    elif outcome == "Loss":
        overall_win.append(np.nan)
        overall_loss.append(outcome)
        win_only.append(np.nan)
        loss_only.append(outcome)
        bust.append(np.nan)
        dealer_bust.append(np.nan)
        blackjack.append(np.nan)
        push.append(np.nan)
    elif outcome == "Bust":
        overall_win.append(np.nan)
        overall_loss.append(outcome)
        win_only.append(np.nan)
        loss_only.append(np.nan)
        bust.append(outcome)
        dealer_bust.append(np.nan)
        blackjack.append(np.nan)
        push.append(np.nan)
    elif outcome == "Dealer Bust":
        overall_win.append(outcome)
        overall_loss.append(np.nan)
        win_only.append(np.nan)
        loss_only.append(np.nan)
        bust.append(np.nan)
        dealer_bust.append(outcome)
        blackjack.append(np.nan)
        push.append(np.nan)
    elif outcome == "Blackjack":
        overall_win.append(outcome)
        overall_loss.append(np.nan)
        win_only.append(np.nan)
        loss_only.append(np.nan)
        bust.append(np.nan)
        dealer_bust.append(np.nan)
        blackjack.append(outcome)
        push.append(np.nan)
    elif outcome == "Push":
        overall_win.append(np.nan)
        overall_loss.append(np.nan)
        win_only.append(np.nan)
        loss_only.append(np.nan)
        bust.append(np.nan)
        dealer_bust.append(np.nan)
        blackjack.append(np.nan)
        push.append(outcome)

df["Overall Win"] = overall_win
df["Overall Loss"] = overall_loss
df["Win Only"] = win_only
df["Loss Only"] = loss_only
df["Bust"] = bust
df["Dealer Bust"] = dealer_bust
df["Blackjack"] = blackjack
df["Push"] = push
df.head(5)

,Current Money,Player Totals,Dealer Totals,Player 1st Card,Player 2nd Card,Dealer 1st Card,Dealer 2nd Card,Hit Count,Bet,Outcome,...,"Hand (1st,2nd / Dealer's)",Player Hand,Overall Win,Overall Loss,Win Only,Loss Only,Bust,Dealer Bust,Blackjack,Push
1028432,-10425.0,22,21,10,2,11,10,1,5,Bust,...,"2,10 / 11","2,10",NaN,Bust,NaN,NaN,Bust,NaN,NaN,NaN
543768,2090.0,15,17,10,2,2,10,1,5,Loss,...,"2,10 / 2","2,10",NaN,Loss,NaN,Loss,NaN,NaN,NaN,NaN
129221,13385.0,20,20,10,2,9,11,1,5,Push,...,"2,10 / 9","2,10",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Push
798623,-2222.5,19,19,2,7,3,6,1,10,Push,...,"2,7 / 3","2,7",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Push
273139,9802.5,12,20,2,10,5,8,0,5,Loss,...,"2,10 / 5","2,10",NaN,Loss,NaN,Loss,NaN,NaN,NaN,NaN


In [201]:
#GROUPBY PLAYER HAND COMBINATIONS
groupby_hand = df.groupby(["Hand (1st,2nd / Dealer's)"]).count()
groupby_hand["Win Only %"] = (groupby_hand["Win Only"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)  #I can divide the win count by any column because I aggregated by count when grouping by, so all other columns are the same. Same with the 5 lines below
groupby_hand["Overall Win %"] = (groupby_hand["Overall Win"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand["Overall Loss %"] = (groupby_hand["Overall Loss"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand["Dealer Bust %"] = (groupby_hand["Dealer Bust"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand["Loss Only %"] = (groupby_hand["Loss Only"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand["Bust %"] = (groupby_hand["Bust"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand["Blackjack %"] = (groupby_hand["Blackjack"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand["Push %"] = (groupby_hand["Push"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
print(f'# of rows: {len(groupby_hand)}')
groupby_hand.reset_index().head(5)

# of rows: 550


,"Hand (1st,2nd / Dealer's)",Current Money,Player Totals,Dealer Totals,Player 1st Card,Player 2nd Card,Dealer 1st Card,Dealer 2nd Card,Hit Count,Bet,...,Push,Initial Hand,Win Only %,Overall Win %,Overall Loss %,Dealer Bust %,Loss Only %,Bust %,Blackjack %,Push %
0,"10,10 / 10",29053,29053,29053,29053,29053,29053,29053,29053,29053,...,9957,29053,33.35%,54.47%,11.26%,21.11%,11.26%,0.00%,0.00%,34.27%
1,"10,10 / 11",7389,7389,7389,7389,7389,7389,7389,7389,7389,...,971,7389,39.32%,51.04%,35.82%,11.72%,35.82%,0.00%,0.00%,13.14%
2,"10,10 / 2",7395,7395,7395,7395,7395,7395,7395,7395,7395,...,961,7395,39.97%,75.09%,11.91%,35.12%,11.91%,0.00%,0.00%,13.00%
3,"10,10 / 3",7251,7251,7251,7251,7251,7251,7251,7251,7251,...,853,7251,39.57%,76.68%,11.56%,37.11%,11.56%,0.00%,0.00%,11.76%
4,"10,10 / 4",7339,7339,7339,7339,7339,7339,7339,7339,7339,...,845,7339,37.63%,77.49%,11.00%,39.86%,11.00%,0.00%,0.00%,11.51%


In [193]:
#GROUPBY SIMPLER INITIAL HAND COMBINATIONS
df["Initial Hand"] = df["Player 1st Card"] + df["Player 2nd Card"]
groupby_simpler_hand = df.groupby(["Initial Hand","Dealer 1st Card"]).count()
groupby_simpler_hand["Win Only %"] = (groupby_simpler_hand["Win Only"]/groupby_simpler_hand["Outcome"]).map('{:,.2%}'.format)  #I can divide the win count by any column because I aggregated by count when grouping by, so all other columns are the same. Same with the 5 lines below
groupby_simpler_hand["Overall Win %"] = (groupby_simpler_hand["Overall Win"]/groupby_simpler_hand["Outcome"]).map('{:,.2%}'.format)
groupby_simpler_hand["Overall Loss %"] = (groupby_simpler_hand["Overall Loss"]/groupby_simpler_hand["Outcome"]).map('{:,.2%}'.format)
groupby_simpler_hand["Dealer Bust %"] = (groupby_simpler_hand["Dealer Bust"]/groupby_simpler_hand["Outcome"]).map('{:,.2%}'.format)
groupby_simpler_hand["Loss Only %"] = (groupby_simpler_hand["Loss Only"]/groupby_simpler_hand["Outcome"]).map('{:,.2%}'.format)
groupby_simpler_hand["Bust %"] = (groupby_simpler_hand["Bust"]/groupby_simpler_hand["Outcome"]).map('{:,.2%}'.format)
groupby_simpler_hand["Blackjack %"] = (groupby_simpler_hand["Blackjack"]/groupby_simpler_hand["Outcome"]).map('{:,.2%}'.format)
groupby_simpler_hand["Push %"] = (groupby_simpler_hand["Push"]/groupby_simpler_hand["Outcome"]).map('{:,.2%}'.format)
#groupby_simpler_hand.reset_index()

In [200]:
#GROUPBY DEALER HAND COMBINATIONS
groupby_dealer_hand = df.groupby(["Dealer 1st Card"]).count()
groupby_dealer_hand["Win Only %"] = (groupby_dealer_hand["Win Only"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)  #I can divide the win count by any column because I aggregated by count when grouping by, so all other columns are the same. Same with the 5 lines below
groupby_dealer_hand["Overall Win %"] = (groupby_dealer_hand["Overall Win"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand["Overall Loss %"] = (groupby_dealer_hand["Overall Loss"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand["Dealer Bust %"] = (groupby_dealer_hand["Dealer Bust"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand["Loss Only %"] = (groupby_dealer_hand["Loss Only"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand["Bust %"] = (groupby_dealer_hand["Bust"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand["Blackjack %"] = (groupby_dealer_hand["Blackjack"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand["Push %"] = (groupby_dealer_hand["Push"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand.reset_index().head(5)

# of rows: 550


,Dealer 1st Card,Current Money,Player Totals,Dealer Totals,Player 1st Card,Player 2nd Card,Dealer 2nd Card,Hit Count,Bet,Outcome,...,Push,Initial Hand,Win Only %,Overall Win %,Overall Loss %,Dealer Bust %,Loss Only %,Bust %,Blackjack %,Push %
0,2,80444,80444,80444,80444,80444,80444,80444,80444,80444,...,5072,80444,11.77%,48.95%,44.74%,32.82%,41.49%,3.25%,4.37%,6.31%
1,3,80224,80224,80224,80224,80224,80224,80224,80224,80224,...,4822,80224,11.37%,50.44%,43.55%,34.68%,40.43%,3.12%,4.40%,6.01%
2,4,80777,80777,80777,80777,80777,80777,80777,80777,80777,...,4246,80777,9.99%,52.35%,42.40%,37.74%,42.40%,0.00%,4.62%,5.26%
3,5,80699,80699,80699,80699,80699,80699,80699,80699,80699,...,4015,80699,9.36%,53.73%,41.29%,39.86%,41.29%,0.00%,4.51%,4.98%
4,6,80631,80631,80631,80631,80631,80631,80631,80631,80631,...,3991,80631,10.23%,55.21%,39.84%,40.46%,39.84%,0.00%,4.51%,4.95%


In [196]:
#EXPORT TO CSV's
groupby_hand.to_csv("blackjack_results.csv")
groupby_dealer_hand.to_csv("blackjack_groupby_dealer.csv")

In [199]:
#AVG PLAYER EDGE (loss/hand)
(df["Current Money"][999990] - df["Current Money"][0])/10000/5

-0.4814

In [198]:
# #GRAPH MONEY OVER TIME
# mychart = df["Current Money"].plot(kind='line')
# mychart.set_xlabel("# of hands")
# mychart.set_ylabel("Money")